In [5]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [7]:
testdf = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../../Benchmark-Labeled-Data/Metadata/meta_data.csv')

test_merged = pd.merge(testdf,test_metadata,on='Record_id')
y_true = test_merged.y_act.values.tolist()

In [ ]:
# We first change the AutoGluon's default_learner's fit function to make sure that we just return the ML feature types rather than building their entire ML pipeline.
# Just replace the fit function with this codeblock in project_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/learner/default_learner.py
# project_env is my python virtual environment directory

#     def fit(self, X: DataFrame, X_test: DataFrame = None, scheduler_options=None, hyperparameter_tune=True,
#             feature_prune=False, holdout_frac=0.1, num_bagging_folds=0, num_bagging_sets=1, stack_ensemble_levels=0,
#             hyperparameters=None, time_limit=None, save_data=False, save_bagged_folds=True, verbosity=2):
#         """ Arguments:
#                 X (DataFrame): training data
#                 X_test (DataFrame): data used for hyperparameter tuning. Note: final model may be trained using this data as well as training data
#                 hyperparameter_tune (bool): whether to tune hyperparameters or simply use default values
#                 feature_prune (bool): whether to perform feature selection
#                 scheduler_options (tuple: (search_strategy, dict): Options for scheduler
#                 holdout_frac (float): Fraction of data to hold out for evaluating validation performance (ignored if X_test != None, ignored if kfolds != 0)
#                 num_bagging_folds (int): kfolds used for bagging of models, roughly increases model training time by a factor of k (0: disabled)
#                 num_bagging_sets (int): number of repeats of kfold bagging to perform (values must be >= 1),
#                     total number of models trained during bagging = num_bagging_folds * num_bagging_sets
#                 stack_ensemble_levels : (int) Number of stacking levels to use in ensemble stacking. Roughly increases model training time by factor of stack_levels+1 (0: disabled)
#                     Default is 0 (disabled). Use values between 1-3 to improve model quality.
#                     Ignored unless kfolds is also set >= 2
#                 hyperparameters (dict): keys = hyperparameters + search-spaces for each type of model we should train.
#         """
#         if hyperparameters is None:
#             hyperparameters = {'NN': {}, 'GBM': {}}
#         # TODO: if provided, feature_types in X, X_test are ignored right now, need to pass to Learner/trainer and update this documentation.
#         if time_limit:
#             self.time_limit = time_limit
#             logger.log(20, f'Beginning AutoGluon training ... Time limit = {time_limit}s')
#         else:
#             self.time_limit = 1e7
#             logger.log(20, 'Beginning AutoGluon training ...')
#         logger.log(20, f'AutoGluon will save models to {self.path}')
#         logger.log(20, f'Train Data Rows:    {len(X)}')
#         logger.log(20, f'Train Data Columns: {len(X.columns)}')
#         if X_test is not None:
#             logger.log(20, f'Tuning Data Rows:    {len(X_test)}')
#             logger.log(20, f'Tuning Data Columns: {len(X_test.columns)}')
#         time_preprocessing_start = time.time()
#         logger.log(20, 'Preprocessing data ...')

#         X_before=X
#         X, y, X_test, y_test, holdout_frac, num_bagging_folds = self.general_data_processing(X, X_test, holdout_frac, num_bagging_folds)
#         time_preprocessing_end = time.time()
#         self.time_fit_preprocessing = time_preprocessing_end - time_preprocessing_start
#         logger.log(20, f'\tData preprocessing and feature engineering runtime = {round(self.time_fit_preprocessing, 2)}s ...')
#         if time_limit:
#             time_limit_trainer = time_limit - self.time_fit_preprocessing
#         else:
#             time_limit_trainer = None

#         df = pd.DataFrame(columns=['column', 'feature_type'])

#         for col in X_before.columns:
#             if col=='label_target': continue
#             df_col = X_before[col]

#             numeric = False
#             if df_col.dtypes == 'int64' or df_col.dtypes == 'float64': numeric = True

#             text = False
#             X_unique = df_col.unique()
#             num_unique = len(X_unique)
#             num_rows = len(df_col)
#             unique_ratio = num_unique / num_rows
#             if unique_ratio <= 0.01: text = False
#             else:
#                 import re
#                 avg_words = np.mean([len(re.sub(' +', ' ', value).split(' ')) if isinstance(value, str) else 0 for value in X_unique])
#                 if avg_words < 3: text = False
#                 else: text = True

#             unusable = False
#             col_val = df_col
#             num_unique = len(col_val.unique())
#             if num_unique == 1: unusable = True

#             date = False
#             try:
#                 df_col.apply(pd.to_datetime)
#                 date = True
#             except: date = False
            
#             curpred=0
#             if unusable: curpred = 7
#             elif text: curpred = 3
#             elif date: curpred = 2
#             if date and numeric and (not unusable): curpred = 0
            
#             if (not numeric) and curpred == 0:
#                 check_unusable_for_categ=False
#                 rank = df_col.value_counts().sort_values(ascending=True)
#                 rank = rank[rank >= 3]
#                 rank = rank.reset_index()
#                 val_list = list(rank['index'].values)
#                 if len(val_list) <= 1: check_unusable_for_categ = True

#                 if check_unusable_for_categ: curpred = 7
#                 else: curpred = 1

#             df.loc[len(df)] = [col, curpred]

#         df.to_csv("AutoGluon_predictions.csv",index=False, mode='a', header=False)
#         return self.feature_generator.features

In [9]:
import copy

def Load_GLUON(dataDownstream):
    df = pd.DataFrame(columns=['column', 'feature_type'])
    df.to_csv('AutoGluon_predictions.csv',index=False)

    import time
    time.sleep(3)
    train= copy.deepcopy(dataDownstream)

    train['label_target'] = 1
    train_data = TabularDataset(train)
    label_column = 'label_target'

    try: features = TabularPredictor(label=label_column).fit(train_data=train_data)
    except: AlwaysTrue = 1
    

    agl_predictions = pd.read_csv('AutoGluon_predictions.csv')
    predictions = agl_predictions['feature_type'].values.tolist()
    
    curpred=0
    if(len(predictions)==0): curpred=7
    else: curpred=predictions[0]
    return curpred
    
    
cntExceptions = 0
y_agl = [0]*1985
prv_csv_name,csv_name = '',''
exception_indices = []

for index, row in test_merged.iterrows():
    if index%100==0:  print(index)
    col = row['Attribute_name']
    prv_csv_name = csv_name
    csv_name = '../../RawCSV/' + row['name']
    
    if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')

    try:
        df_col = df[[col]]
    except KeyError:
        y_agl[index]=1
        exception_indices.append(row)
        cntExceptions += 1
        continue
        
    y_agl[index] = Load_GLUON(df_col)


0


No path specified. Models will be saved in: "AutogluonModels/ag-20231011_205153/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231011_205153/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #33~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Sep  7 10:33:52 UTC 2
Disk Space Avail:   829.06 GB / 997.61 GB (83.1%)
Train Data Rows:    1781
Train Data Columns: 1
Label Column: label_target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	1 unique label values:  [1]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 1
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGe

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_true = [dict_label_true[str(i)] for i in y_true]

print(accuracy_score(y_true, y_agl))
print(confusion_matrix(y_true, y_agl))